In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tqdm.notebook import tqdm

from src import visualisation

## Load data

In [2]:
results = visualisation.Results("data/raw_results")

## Demo

In [3]:
network = "l2_course_net_1"
budget = 15
protocol = "AND"
mi_value = 0.10
ss_method = "random"

In [ ]:
r_slice_nml = results.get_slice(
    protocol=protocol,
    mi_value=mi_value,
    seed_budget=budget,
    network=network,
    ss_method=ss_method,
)
r_slice_nml

In [ ]:
r_slice_mds = results.get_slice(
    protocol=protocol,
    mi_value=mi_value,
    seed_budget=budget,
    network=network,
    ss_method=f"d^{ss_method}",
)
r_slice_mds

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(2, 1.5))  # budget x mi
visualisation.Plotter().plot_single_comparison(
    record_mds=results.mean_expositions_rec(r_slice_mds),
    record_nml=results.mean_expositions_rec(r_slice_nml),
    actors_nb=results.get_actors_nb(r_slice_mds),
    mi_value=mi_value,
    seed_budget=budget,
    ax=ax
)

### Plot visualisaitons to PDF

In [7]:
workdir = Path("data/raw_results")
workdir.mkdir(exist_ok=True, parents=True)

plotter = visualisation.Plotter()
pdf = PdfPages(workdir.joinpath(f"expositions.pdf"))

In [ ]:
for page_idx, page_case in enumerate(plotter.yield_page()):
    print(page_case)

    fig, axs = plt.subplots(
        nrows=len(plotter._seed_budgets_and if page_case[1] == "AND" else plotter._seed_budgets_and),
        ncols=len(plotter._mi_values),
        figsize=(15, 20),
    )

    for fig_idx, fig_case in tqdm(enumerate(plotter.yield_figure(protocol=page_case[1]))):
        row_idx = fig_idx // len(axs[0])
        col_idx = fig_idx % len(axs[1])
        # print(page_case, fig_case, page_idx, row_idx, col_idx)

        nml_slice = results.get_slice(
            protocol=page_case[1],
            mi_value=fig_case[1],
            seed_budget=fig_case[0],
            network=page_case[0],
            ss_method=page_case[2],
        )
        mds_slice = results.get_slice(
            protocol=page_case[1],
            mi_value=fig_case[1],
            seed_budget=fig_case[0],
            network=page_case[0],
            ss_method=f"d^{page_case[2]}",
        )
        if len(nml_slice) == 0 or len(mds_slice) == 0:
            plotter.plot_dummy_fig(
                mi_value=fig_case[1],
                seed_budget=fig_case[0],
                ax=axs[row_idx][col_idx],
            )
        else:
            plotter.plot_single_comparison(
                record_mds=results.mean_expositions_rec(mds_slice),
                record_nml=results.mean_expositions_rec(nml_slice),
                actors_nb=results.get_actors_nb(nml_slice),
                mi_value=fig_case[1],
                seed_budget=fig_case[0],
                ax=axs[row_idx][col_idx],
            )
    
    fig.tight_layout(pad=.5, rect=(0.05, 0.05, 0.95, 0.95))
    fig.suptitle(f"Network: {page_case[0]}, Protocol: {page_case[1]}, SSM: {page_case[2]}")
    fig.savefig(pdf, format="pdf")
    plt.close(fig)

pdf.close()